In [ ]:
# https://towardsdatascience.com/quickly-test-multiple-models-a98477476f0

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
import plotly.express as px
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [ ]:
threshold = 1
random_state = 42

In [ ]:
# Import data and preprocess
df = pd.read_csv("../test_data/Grounded CPW Leaky Wave antenna/S11 Data.csv")
df = df.drop(df[df['dB(S(1,1)) []'] > 0].index) # Remove all rows with positive s11

# Split into x and y
input_x = df.drop(columns=['dB(S(1,1)) []'], axis=1)
input_y = df[['dB(S(1,1)) []']]

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(input_x, input_y, random_state=random_state)

In [ ]:
def generate_pipeline(scaler, model):
    return (model.__class__.__name__, Pipeline(steps=[('normalize', scaler), ('model', model)]))

In [ ]:
# Check if predicted value is threshold amount above or below actual value
def is_in_threshold(actual, pred):
    return pred <= actual + threshold and pred >= actual - threshold

def create_tf_column(results):
    return results.apply(lambda x: is_in_threshold(x['y_test'], x['predictions']), axis=1)

def get_score(y_test, y_pred, **kwargs):
    dataframe = pd.DataFrame(y_test, columns=['y_test'])
    dataframe['predictions'] = y_pred
    return create_tf_column(dataframe).value_counts().get(True) / dataframe.shape[0]

# def get_score_singular(y_test, y_pred):
#     return get_score(y_test, y_pred)

def get_rsme(y_test, y_pred):
    return mean_squared_error(y_test, y_pred, squared=False)

def get_r_squared(y_test, y_pred):
    return r2_score(y_test, y_pred)

In [ ]:
custom_scorer = make_scorer(get_score, greater_is_better=True)
scaler = MinMaxScaler(feature_range=(0,1)) # Initialize scaler

In [ ]:
rfr_model = RandomForestRegressor(random_state=random_state)
rfr_pipeline = Pipeline(steps=[('normalize', scaler), ('model', rfr_model)])

random_grid = {'model__n_estimators': [int(x) for x in np.linspace(start=100, stop=1000, num=5)],
               'model__max_depth': [int(x) for x in np.linspace(10, 110, num=5)],
               'model__min_samples_split': [2, 5, 10],
               'model__min_samples_leaf': [1, 2, 4]}

random_forest_search = RandomizedSearchCV(rfr_pipeline, random_grid, n_iter=20, scoring=custom_scorer, cv=10, verbose=10, n_jobs=-1)
random_forest_search.fit(input_x, input_y.values.ravel())
print(random_forest_search.best_params_)

In [285]:
gbr_model = GradientBoostingRegressor(random_state=random_state)
gbr_pipeline = Pipeline(steps=[('normalize', scaler), ('model', gbr_model)])

random_grid = {'model__learning_rate': [0.001],
               'model__n_estimators': [1000, 3000, 5000, 7000, 10000],
               'model__max_depth': [1, 2, 3, 5, 7, 10]}

gradient_boosting_search = RandomizedSearchCV(gbr_pipeline, random_grid, n_iter=20, scoring=custom_scorer, cv=10, verbose=10, n_jobs=-1)
gradient_boosting_search.fit(input_x, input_y.values.ravel())
print(gradient_boosting_search.best_params_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV 2/10; 1/20] START model__learning_rate=0.001, model__max_depth=3, model__n_estimators=3000
[CV 1/10; 1/20] START model__learning_rate=0.001, model__max_depth=3, model__n_estimators=3000
[CV 3/10; 1/20] START model__learning_rate=0.001, model__max_depth=3, model__n_estimators=3000
[CV 5/10; 1/20] START model__learning_rate=0.001, model__max_depth=3, model__n_estimators=3000
[CV 4/10; 1/20] START model__learning_rate=0.001, model__max_depth=3, model__n_estimators=3000
[CV 7/10; 1/20] START model__learning_rate=0.001, model__max_depth=3, model__n_estimators=3000
[CV 6/10; 1/20] START model__learning_rate=0.001, model__max_depth=3, model__n_estimators=3000
[CV 8/10; 1/20] START model__learning_rate=0.001, model__max_depth=3, model__n_estimators=3000
[CV 9/10; 1/20] START model__learning_rate=0.001, model__max_depth=3, model__n_estimators=3000
[CV 1/10; 2/20] START model__learning_rate=0.001, model__max_depth=2, model__n_est

/home/meow/.local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.




[CV 2/10; 8/20] END model__learning_rate=0.001, model__max_depth=10, model__n_estimators=5000;, score=0.480 total time=  24.6s
[CV 3/10; 8/20] START model__learning_rate=0.001, model__max_depth=10, model__n_estimators=5000
[CV 4/10; 8/20] START model__learning_rate=0.001, model__max_depth=10, model__n_estimators=5000
[CV 5/10; 8/20] START model__learning_rate=0.001, model__max_depth=10, model__n_estimators=5000
[CV 6/10; 8/20] START model__learning_rate=0.001, model__max_depth=10, model__n_estimators=5000
[CV 5/10; 7/20] END model__learning_rate=0.001, model__max_depth=10, model__n_estimators=10000;, score=0.393 total time=  48.3s
[CV 3/10; 7/20] END model__learning_rate=0.001, model__max_depth=10, model__n_estimators=10000;, score=0.639 total time=  49.2s
[CV 7/10; 8/20] START model__learning_rate=0.001, model__max_depth=10, model__n_estimators=5000
[CV 8/10; 8/20] START model__learning_rate=0.001, model__max_depth=10, model__n_estimators=5000
[CV 4/10; 7/20] END model__learning_rate

In [295]:
svr_model = SVR()
svr_pipeline = Pipeline(steps=[('normalize', scaler), ('model', svr_model)])

random_grid = {
    'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'model__C': [1,5,10],
    'model__degree': [3,8],
    'model__coef0': [0.01,10,0.5],
    'model__gamma': ['auto','scale']
}
svr_search = RandomizedSearchCV(svr_pipeline, random_grid, n_iter=20, cv=10, verbose=10, scoring=custom_scorer, n_jobs=-1)
svr_search.fit(input_x, input_y.values.ravel())
print(svr_search.best_params_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV 1/10; 1/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=sigmoid
[CV 7/10; 1/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=sigmoid
[CV 2/10; 1/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=sigmoid
[CV 5/10; 1/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=sigmoid
[CV 3/10; 1/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=sigmoid
[CV 8/10; 1/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=sigmoid
[CV 9/10; 1/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=sigmoid
[CV 10/10; 1/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=sigmoid
[CV 6/10; 1/20] START model__C=5, model__coef0=10, model__degree

/home/meow/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/tmp/ipykernel_230/1665397672.py", line 11, in get_score
TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

  warnings.warn(


[CV 1/10; 10/20] END model__C=1, model__coef0=0.01, model__degree=8, model__gamma=scale, model__kernel=rbf;, score=0.192 total time=   1.0s
[CV 10/10; 10/20] START model__C=1, model__coef0=0.01, model__degree=8, model__gamma=scale, model__kernel=rbf
[CV 8/10; 9/20] END model__C=1, model__coef0=0.01, model__degree=3, model__gamma=scale, model__kernel=sigmoid;, score=0.019 total time=   1.3s
[CV 1/10; 11/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=rbf
[CV 2/10; 10/20] END model__C=1, model__coef0=0.01, model__degree=8, model__gamma=scale, model__kernel=rbf;, score=0.147 total time=   1.0s
[CV 2/10; 11/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=rbf
[CV 9/10; 9/20] END model__C=1, model__coef0=0.01, model__degree=3, model__gamma=scale, model__kernel=sigmoid;, score=0.005 total time=   1.3s
[CV 3/10; 11/20] START model__C=5, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=rbf
[CV 10

/home/meow/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/tmp/ipykernel_230/1665397672.py", line 11, in get_score
TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

  warnings.warn(


[CV 1/10; 18/20] END model__C=10, model__coef0=0.01, model__degree=8, model__gamma=scale, model__kernel=sigmoid;, score=nan total time=   1.6s
[CV 3/10; 18/20] START model__C=10, model__coef0=0.01, model__degree=8, model__gamma=scale, model__kernel=sigmoid


...

[CV 2/10; 13/20] END model__C=10, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=poly;, score=nan total time= 4.4min
[CV 7/10; 20/20] START model__C=5, model__coef0=10, model__degree=3, model__gamma=auto, model__kernel=linear
[CV 6/10; 20/20] END model__C=5, model__coef0=10, model__degree=3, model__gamma=auto, model__kernel=linear;, score=0.202 total time=   0.7s
[CV 8/10; 20/20] START model__C=5, model__coef0=10, model__degree=3, model__gamma=auto, model__kernel=linear
[CV 7/10; 20/20] END model__C=5, model__coef0=10, model__degree=3, model__gamma=auto, model__kernel=linear;, score=0.157 total time=   0.7s
[CV 9/10; 20/20] START model__C=5, model__coef0=10, model__degree=3, model__gamma=auto, model__kernel=linear
[CV 8/10; 20/20] END model__C=5, model__coef0=10, model__degree=3, model__gamma=auto, model__kernel=linear;, score=0.236 total time=   0.8s
[CV 10/10; 20/20] START model__C=5, model__coef0=10, model__degree=3, model__gamma=auto, model__kernel=linear
[CV 9/

/home/meow/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/tmp/ipykernel_230/1665397672.py", line 11, in get_score
TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

  warnings.warn(


[CV 3/10; 13/20] END model__C=10, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=poly;, score=nan total time= 4.7min
[CV 7/10; 19/20] END model__C=10, model__coef0=0.5, model__degree=8, model__gamma=scale, model__kernel=poly;, score=0.119 total time= 2.4min
[CV 9/10; 19/20] END model__C=10, model__coef0=0.5, model__degree=8, model__gamma=scale, model__kernel=poly;, score=0.076 total time= 1.7min
[CV 10/10; 19/20] END model__C=10, model__coef0=0.5, model__degree=8, model__gamma=scale, model__kernel=poly;, score=0.164 total time= 1.2min


/home/meow/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/home/meow/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/tmp/ipykernel_230/1665397672.py", line 11, in get_score
TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

  warnings.warn(
/home/meow/.local/lib/python3.10/site-packages/sklearn/model_selection/_

[CV 7/10; 13/20] END model__C=10, model__coef0=10, model__degree=8, model__gamma=auto, model__kernel=poly;, score=nan total time= 6.0min
{'model__kernel': 'poly', 'model__gamma': 'scale', 'model__degree': 3, 'model__coef0': 0.5, 'model__C': 1}


In [298]:
scaler = MinMaxScaler(feature_range=(0,1))
models = [generate_pipeline(scaler, model) for model in [
    CatBoostRegressor(),
    XGBRegressor(),
    SVR(kernel='poly', gamma='scale', degree=3, coef0=0.5, C=1),
    GradientBoostingRegressor(n_estimators=10000, max_depth=10, learning_rate=0.001, random_state=random_state),
    RandomForestRegressor(max_depth=110, min_samples_leaf=1, min_samples_split=5, n_estimators=1000, random_state=random_state)
]]

In [300]:
# Test the random forest model 
model = models[-3]
clf = model[1].fit(X_train, y_train.values.ravel())
y_pred = clf.predict(X_test)
get_score(y_test.values.ravel(), y_pred)

0.23596574690770694

In [301]:
pd.DataFrame({'y_actual': y_test.values.reshape(-1), 'y_pred': y_pred})

,y_actual,y_pred
0,-10.378239,-5.304665
1,-6.207769,-10.128150
2,-12.617678,-12.308452
3,-0.352397,-1.781749
4,-5.473230,-5.230557
...,...,...
1046,-7.475370,-8.493637
1047,-5.701183,-5.939416
1048,-4.898163,-6.717723
1049,-11.964244,-7.489813


In [302]:
results = {}

for name, model in models:
  clf = model.fit(X_train, y_train.values.ravel())
  y_pred = clf.predict(X_test)

  results[name] = {
    'score': get_score(y_test.values.ravel(), y_pred),
    'rsme': get_rsme(y_test.values.ravel(), y_pred),
    'r_squared': get_r_squared(y_test.values.ravel(), y_pred),
  }

Learning rate set to 0.049086
0:	learn: 6.4309836	total: 2.94ms	remaining: 2.94s
1:	learn: 6.2483283	total: 4.16ms	remaining: 2.08s
2:	learn: 6.0930776	total: 5.28ms	remaining: 1.75s
3:	learn: 5.9458282	total: 6.66ms	remaining: 1.66s
4:	learn: 5.7976874	total: 7.9ms	remaining: 1.57s
5:	learn: 5.6604455	total: 9.48ms	remaining: 1.57s
6:	learn: 5.5477821	total: 10.7ms	remaining: 1.51s
7:	learn: 5.4277746	total: 11.7ms	remaining: 1.45s
8:	learn: 5.3150829	total: 12.7ms	remaining: 1.4s
9:	learn: 5.2083583	total: 13.8ms	remaining: 1.36s
10:	learn: 5.1127149	total: 14.9ms	remaining: 1.34s
11:	learn: 5.0256453	total: 16.1ms	remaining: 1.32s
12:	learn: 4.9368142	total: 17.6ms	remaining: 1.33s
13:	learn: 4.8638618	total: 18.7ms	remaining: 1.31s
14:	learn: 4.7891263	total: 19.7ms	remaining: 1.29s
15:	learn: 4.7202463	total: 21.5ms	remaining: 1.32s
16:	learn: 4.6574763	total: 22.8ms	remaining: 1.32s
17:	learn: 4.5962597	total: 23.9ms	remaining: 1.3s
18:	learn: 4.5382782	total: 25.3ms	remaining: 1

In [303]:
df = pd.DataFrame(results).T

In [304]:
px.bar(df, x=df.index, y='score')

In [305]:
px.bar(df, x=df.index, y='rsme')

In [306]:
px.bar(df, x=df.index, y='r_squared')